In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from PIL import ImageGrab
from datetime import datetime
import os,csv,pandas as pd, numpy as np, pyautogui as at
import requests,json,time,bs4,hashlib

In [6]:
def getSettingsProgram():
    datas = open("./dataProgram/setting.txt",'r',encoding="utf8")
    result = {}
    for data in datas.readlines():
        splitdatas = data.split(',,,')
        for i in splitdatas:
            j = i.split('=')
            if(len(j)==2):
                if(j[0] == "positionDowload" or j[0] == "position_img"):
                    g = j[1].split(',')
                    result[j[0]] = (int(g[0]),int(g[1]))
                    # result[j[0]] = {'x':int(g[0]),'y':int(g[1])}
                else:
                    result[j[0]] = j[1]
    return result

In [9]:
extension_path1 = './LogPrograme/3.0.8_0.crx'
extension_path2 = './LogPrograme/13.0.10_0.crx'
download_path1 = getSettingsProgram()["pathdownload"]
download_path = download_path1.replace("/","\\")
uriAPI = getSettingsProgram()["urlAPI"]
position_img = getSettingsProgram()["position_img"]
downcount = getSettingsProgram()["downcount"]
timesleep20item = getSettingsProgram()["timesleep20item"]
positionDowload = (1133,289) 
timesleepLoadimage = getSettingsProgram()["timesleepLoadimage"]
timesleepLoadSheach = getSettingsProgram()["timesleepLoadSheach"]
timesleepReadCSV = getSettingsProgram()["timesleepReadCSV"]

In [24]:
web_options = webdriver.ChromeOptions()
web_options.add_extension(extension_path1)
web_options.add_extension(extension_path2)
web_options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
})
driver = webdriver.Chrome(options=web_options)
driver.get('https://www.aliprice.com/Member/login.html?ext_id=10100&platform=1688&version=3.0.8&browser=chrome&channel=chrome&mv=3&redirect=https%3A%2F%2Fwww.1688.com')
# driver.window_handles
driver.switch_to.window(driver.window_handles[-1])
driver.execute_script("window.close();")
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])
driver.execute_script("window.close();")
driver.switch_to.window(driver.window_handles[0])
# Login 1688
# email : tmhumr1frz2vjv7nf1i62lc@sharklasers.com
# password : passapol47
urlserver = 'http://localhost:3030/'

In [10]:
groups = {
	"อุปกรณ์เสริม-อิเล็กทรอนิกส์":{"thai":"อุปกรณ์เสริม-อิเล็กทรอนิกส์","eng":"group1"},
	"อุปกรณ์-อิเล็กทรอนิกส์":{"thai":"อุปกรณ์-อิเล็กทรอนิกส์","eng":"group2"},
	"ทีวีและเครื่องใช้ในบ้าน":{"thai":"ทีวีและเครื่องใช้ในบ้าน","eng":"group3"},
	"สุขภาพและความงาม":{"thai":"สุขภาพและความงาม","eng":"group4"},
	"ทารกและของเล่น":{"thai":"ทารกและของเล่น","eng":"group5"},
	"ของชำและสัตว์เลี้ยง":{"thai":"ของชำและสัตว์เลี้ยง","eng":"group6"},
	"บ้านและไลฟ์สไตล์":{"thai":"บ้านและไลฟ์สไตล์","eng":"group7"},
	"แฟชั่นและเครื่องประดับผู้หญิง":{"thai":"แฟชั่นและเครื่องประดับผู้หญิง","eng":"group8"},
	"แฟชั่นและเครื่องประดับผู้ชาย":{"thai":"แฟชั่นและเครื่องประดับผู้ชาย","eng":"group9"},
	"กีฬาและการเดินทาง":{"thai":"กีฬาและการเดินทาง","eng":"group10"},
	"ยานยนต์และรถจักรยานยนต์":{"thai":"ยานยนต์และรถจักรยานยนต์","eng":"group11"},
	"เครื่อเขียนหนังสือ":{"thai":"เครื่อเขียนหนังสือ","eng":"group12"},
	"เครื่องประดับ":{"thai":"เครื่องประดับ","eng":"group13"},
	"ตั๋วและบัตรกำนัน":{"thai":"ตั๋วและบัตรกำนัน","eng":"group14"},
}

CNY_TO_THB = 5.30
url = "https://api.exchangerate-api.com/v4/latest/CNY"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    CNY_TO_THB = data["rates"]["THB"]

def sha256(data):
    return hashlib.sha256(data.encode()).hexdigest()
def setSettingsProgram(pathFile,newSetting):
    datas = open(pathFile,'w',encoding="utf8")
    datas.write(newSetting)
    datas.close()
def readCsvAll(folder_path='./'):
    csv_files = []
    try:
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                csv_files.append(filename)
    except:pass
    return csv_files
def readFileCsv(name):
    dataCsv = []
    # try:
    with open(name, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            dataCsv.append(row)
    # except:pass
    return dataCsv
def postDataAPI1688(data,group,idproduct):
    uri = f"{uriAPI}postdataby1688?key=5a3dec84301206e275f7ca7fa119796c8a5be05d100a2d23ba3a4f189876d03a"
    res = requests.post(uri,data={
        "item20":data,
        "group":group,
        "idproduct":idproduct
    })
    return res
def formatHeader(i):
    key = "";
    if("ชื่อร้าน" in i):
        key = "namestore"
    elif("ชื่อผลิตภัณฑ์" in i):
        key = "nameproduct"
    elif("ราคา" in i):
        key = "priceCNY"
    elif("ลิงค์ผลิตภัณฑ์" in i):
        key = "linkproduct"
    elif("ลิงค์รูปภาพ" in i):
        key = "linkimage"
    elif("จำนวนคำสั่งชำระเงินใน 30 วันที่ผ่านมา" in i):
        key = "countpayment30daylater"
    elif("อัตราการฟื้นตัว 48H ในช่วง 30 วันที่ผ่านมา" in i):
        key = "lateheal30daylater"
    elif("อัตราประสิทธิภาพ 48H ในช่วง 30 วันที่ผ่านมา" in i):
        key = "efficient30daylater"
    elif("อัตราการตอบกลับ 3 นาทีในช่วง 30 วันที่ผ่านมา" in i):
        key = "reply3minute30daylater"
    elif("อัตราการคืนเงินที่มีคุณภาพใน 30 วันที่ผ่านมา" in i):
        key = "refund30daylater"
    elif("อัตราการโต้แย้งใน 30 วันที่ผ่านมา" in i):
        key = "argue30daylater"
    elif("การรับรองความแข็งแกร่ง 1688" in i):
        key = "1688late"
    elif("บริการทั่วไป" in i):
        key = "aboutsevice"
    elif("อัตราผลตอบแทน" in i):
        key = "reward"
    elif("แลกเปลี่ยนประสบการณ์" in i):
        key = "experience"
    elif("ประสบการณ์ที่มีคุณภาพ" in i):
        key = "quality"
    elif("ความตรงต่อเวลาของโลจิสติกส์" in i):
        key = "deliveredtime"
    elif("การระงับข้อพิพาท" in i):
        key = "A1"
    elif("ที่ปรึกษาการจัดซื้อจัดจ้าง" in i):
        key = "A2"
    elif("ลักษณะการลงทะเบียน" in i):
        key = "A3"
    elif("ช่วงเข้า" in i):
        key = "AM"
    elif("รูปแบบธุรกิจ" in i):
        key = "typebusiness"
    elif("ที่ตั้ง" in i):
        key = "addresshere"
    elif("ที่อยู่" in i):
        key = "addresslive"
    elif("ข้อมูลติดต่อ" in i):
        key = "teldata"
    elif("ระยะขอบ" in i):
        key = "margin"
    elif("ที่จัดตั้งขึ้น" in i):
        key = "esabished"
    elif("พื้นที่องค์กร" in i):
        key = "areaorganization"
    elif("จำนวนพนักงาน" in i):
        key = "countemployee"
    else:
        key = i
    return key
def saveScreenShot(pathFile):
    im = ImageGrab.grab()
    im.save(pathFile)
def sendMessageAndImageLine(title,pathFile):
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H-%M-%S")
    LINE_ACCESS_TOKEN="kQlzp9toM8Js213IiJCys6qV4cwOuXcXFmp6UcwzgmK"
    url = "https://notify-api.line.me/api/notify"
    file = {'imageFile':open(pathFile,'rb')}
    data = ({
            'message':f'บอท 1688 หยุดทำงาน\n{title}{date_str}{time_str}'
        })
    LINE_HEADERS = {"Authorization":"Bearer "+LINE_ACCESS_TOKEN}
    session = requests.Session()
    r=session.post(url, headers=LINE_HEADERS, files=file, data=data)
def sendMessageLine(title):
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H-%M-%S")
    LINE_ACCESS_TOKEN="kQlzp9toM8Js213IiJCys6qV4cwOuXcXFmp6UcwzgmK"
    url = "https://notify-api.line.me/api/notify"
    data = ({
        'message':f'บอท 1688 ทำงานสำเร็จ\n{title} ของ {date_str}{time_str}'
    })
    LINE_HEADERS = {"Authorization":"Bearer "+LINE_ACCESS_TOKEN}
    session = requests.Session()
    r=session.post(url, headers=LINE_HEADERS, data=data)
def WarningError(title):
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H-%M-%S")
    pathFile = './imageWarning/{}{}{}.png'.format(title,date_str,time_str)
    saveScreenShot(pathFile)
    sendMessageAndImageLine(title,pathFile)
# ///////////////////////////////////////////////
def systemSendAPI(dataInLog):
    filesCSV = readCsvAll(download_path1)
    try:
        df = readFileCsv(download_path1+'/'+filesCSV[-1])
        header = df[0]
        sumproduct = []
        sumid = {}
        for datacsv in df[1::]:
            obj = {}
            for row in range(len(header)):
                obj[formatHeader(header[row])] =  datacsv[row]
            obj["priceCNY"] = float(obj["priceCNY"].split('¥')[1])
            obj["priceTHB"] = float(CNY_TO_THB*float(obj["priceCNY"]))
            obj["id"] = sha256(obj['linkproduct'])
            sumid[sha256(obj['linkproduct'])] = obj
            sumproduct.append(obj)
        postDataAPI1688(json.dumps(sumid),dataInLog["groups"],dataInLog["id"])
        return 200
    except:
        return 404
def fetchAPI(selectGroup,page):
    key = "382557d3731870efb6e863c174ccb351f191987e79a9c5805e16698e055b4364";
    uri = f"{uriAPI}getdatabybot1688?key={key}&&group={selectGroup}&&page={page}"
    try:
        return requests.get(uri);
    except:
        return 404


In [19]:
filesCSV = readCsvAll(download_path1)
df = readFileCsv(download_path1+'/'+filesCSV[-1])
header = df[0]
sumproduct = []
sumid = {}
for datacsv in df[1::]:
    obj = {}
    for row in range(len(header)):
        obj[formatHeader(header[row])] =  datacsv[row]
    obj["priceTHB"] = CNY_TO_THB*float(obj["priceCNY"].split('¥')[1])
    obj["id"] = sha256(obj['linkproduct'])
    sumid[sha256(obj['linkproduct'])] = obj
    sumproduct.append(obj)
res = postDataAPI1688(json.dumps(sumid),"อุปกรณ์เสริม-อิเล็กทรอนิกส์","5961dde36583441759697d291c40b8a073c37c4b8b258c50352dcd70fc0da414")
print(res)

<Response [200]>
